In [1]:
""" 
examine why brd list contents didn't all match up to
SEC > 100M  annual reports
"""

" examine why brd list contents didn't all match up t\nSEC > 100M  annual reports"

In [5]:
import psycopg2 as pg
import pandas.io.sql as pd_sql
import pandas as pd
import numpy as np
import datetime


In [6]:
# Postgres info to connect

connection_args = {
    'host': 'localhost',  
    'dbname': 'bankruptcy',  
    'port': 5432          
}

connection = pg.connect(**connection_args)
connection.autocommit = True   
                              

In [7]:
# few fields for first initial model
# all 2019 data held out for test set

query = """
SELECT *
FROM sec_wide_table
WHERE  period_end_date < '2019-01-01' 
;
"""

sec_data = pd_sql.read_sql(query, connection)
sec_data.head()

,company_name,period_end_date,submission_number,central_index_key,ein,sic,fiscal_year_end,fiscal_year,form,date_filed,...,OperatingIncomeLoss,ProfitLoss,RepaymentsOfLongTermDebt,Revenues,SalesRevenueNet,StockholdersEquity,TotalAsset,Industry,div_code,Division
0,"'MKTG, INC.'",2014-03-31 00:00:00.000000,0001019056-14-000881,886475,061340408,7310,331,2013,10-K,2014-06-27 00:00:00.000000,...,2830011.0,2293272.0,NaN,NaN,NaN,12190322.0,NaN,Services-Advertising,73,Services
1,"024 PHARMA, INC.",2015-12-31 00:00:00.000000,0001683168-17-000653,1307969,201862731,3089,1231,2016,10-K,2017-03-24 00:00:00.000000,...,NaN,NaN,NaN,NaN,NaN,137994.0,NaN,"Plastics Products, NEC",30,Manufacturing
2,"024 PHARMA, INC.",2016-12-31 00:00:00.000000,0001683168-17-000653,1307969,201862731,3089,1231,2016,10-K,2017-03-24 00:00:00.000000,...,NaN,NaN,NaN,1079541.0,NaN,759035.0,NaN,"Plastics Products, NEC",30,Manufacturing
3,1 800 FLOWERS COM INC,2014-06-30 00:00:00.000000,0001437749-14-016921,1084869,113117311,5990,630,2014,10-K,2014-09-12 00:00:00.000000,...,23706000.0,14675000.0,NaN,NaN,756345000.0,183199000.0,NaN,"Retail-Retail Stores, NEC",59,Retail Trade
4,1 800 FLOWERS COM INC,2014-06-30 00:00:00.000000,0001437749-15-017184,1084869,113117311,5990,630,2015,10-K,2015-09-11 00:00:00.000000,...,23706000.0,14675000.0,NaN,NaN,756345000.0,183228000.0,NaN,"Retail-Retail Stores, NEC",59,Retail Trade


In [16]:
company_unique = pd.DataFrame(sec_data['company_name'].unique(), columns={'Name'})

In [27]:
company_unique = set(sec_data['company_name'].unique())
company_unique

{'NEXT 1 INTERACTIVE, INC.',
 'TUNGSTEN CORP.',
 'FORTUNESWELL CORP',
 'TETRIDYN SOLUTIONS INC',
 'UPPERSOLUTION.COM',
 'USD PARTNERS LP',
 'NEWMARK GROUP, INC.',
 'REBEL GROUP, INC.',
 'RAND WORLDWIDE INC',
 'MONOPAR THERAPEUTICS',
 'TRANSATLANTIC CAPITAL INC.',
 'SWIFT START CORP.',
 'DIAMONDHEAD CASINO CORP',
 'FUTURES PORTFOLIO FUND L.P.',
 'STAR BUFFET INC',
 "BJ'S WHOLESALE CLUB HOLDINGS, INC.",
 'HEALTHIENT, INC.',
 'WEST END INDIANA BANCSHARES, INC.',
 'ERA GROUP INC.',
 'HEWLETT PACKARD ENTERPRISE CO',
 'HIGH DESERT HOLDING CORP.',
 'TRANSCAT INC',
 'GREY CLOAK TECH INC.',
 'SUMMIT NETWORKS INC.',
 'IMAGE SENSING SYSTEMS INC',
 'C&J ENERGY SERVICES LTD.',
 'MANNING & NAPIER, INC.',
 'KNOW LABS, INC.',
 'QUEST DIAGNOSTICS INC',
 'TOA HOLDINGS, INC.',
 'COMPX INTERNATIONAL INC',
 'ENERGY XXI LTD',
 'PROFIT PLANNERS MANAGEMENT, INC.',
 'DAIS CORP',
 'LOLLIPOP CORP',
 'MEDIACOM BROADBAND LLC',
 'INDIA ECOMMERCE CORP',
 'STREAMLINE HEALTH SOLUTIONS INC.',
 'FINGERMOTION, INC.',
 'D

In [70]:
#brd_list
brd_labels = pd.read_excel('debtor_list_ein_lookup.xlsx', sheet_name='LIST')
brd_labels.head()

,debtor_name,date_filed,chapter_filing,ein,name_in_sec_data,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,"Caesars Entertainment Operating Company, Inc.",2015-01-12,Chapter 11,621411755.0,CAESARS ENTERTAINMENT CORP,NaN,NaN,NaN,NaN
1,RadioShack Corporation,2015-02-05,Chapter 11,751047710.0,RADIOSHACK CORP,NaN,NaN,NaN,NaN
2,"Cal Dive International, Inc.",2015-03-03,Chapter 11,611500501.0,"Cal Dive International, Inc.",NaN,NaN,NaN,NaN
3,"BPZ Resources, Inc.",2015-03-09,Chapter 11,330502730.0,"BPZ Resources, Inc.",NaN,NaN,NaN,NaN
4,Allied Nevada Gold Corp.,2015-03-10,Chapter 11,0.0,Allied Nevada Gold Corp.,NaN,NaN,NaN,NaN


In [71]:
#capitalize 
brd_labels['company_name_brd'] = brd_labels['name_in_sec_data'].str.upper()
brd_labels.head()

,debtor_name,date_filed,chapter_filing,ein,name_in_sec_data,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,company_name_brd
0,"Caesars Entertainment Operating Company, Inc.",2015-01-12,Chapter 11,621411755.0,CAESARS ENTERTAINMENT CORP,NaN,NaN,NaN,NaN,CAESARS ENTERTAINMENT CORP
1,RadioShack Corporation,2015-02-05,Chapter 11,751047710.0,RADIOSHACK CORP,NaN,NaN,NaN,NaN,RADIOSHACK CORP
2,"Cal Dive International, Inc.",2015-03-03,Chapter 11,611500501.0,"Cal Dive International, Inc.",NaN,NaN,NaN,NaN,"CAL DIVE INTERNATIONAL, INC."
3,"BPZ Resources, Inc.",2015-03-09,Chapter 11,330502730.0,"BPZ Resources, Inc.",NaN,NaN,NaN,NaN,"BPZ RESOURCES, INC."
4,Allied Nevada Gold Corp.,2015-03-10,Chapter 11,0.0,Allied Nevada Gold Corp.,NaN,NaN,NaN,NaN,ALLIED NEVADA GOLD CORP.


In [72]:
brd_names = set(brd_labels['company_name_brd'])
brd_names

{'21ST CENTURY ONCOLOGY HOLDINGS, INC.',
 'ACETO CORP',
 'ADEPTUS HEALTH INC.',
 'AEROPOSTALE INC',
 'ALLIED NEVADA GOLD CORP.',
 'ALPHA NATURAL RESOURCES, INC.',
 'ALTA MESA HOLDINGS, LP',
 'ALTA MESA RESOURCES, INC. /DE',
 'AMERICAN APPAREL, INC',
 'APPROACH RESOURCES INC',
 'ARALEZ PHARMACEUTICALS INC.',
 'ARCH COAL INC',
 'ARMSTRONG ENERGY, INC.',
 'ASPECT SOFTWARE PARENT, INC.',
 'ATLAS RESOURCE PARTNERS, L.P.',
 'AVAYA INC',
 'AZURE MIDSTREAM PARTNERS, LP',
 'BASIC ENERGY SERVICES INC',
 'BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC',
 'BON TON STORES INC',
 'BONANZA CREEK ENERGY, INC.',
 'BPZ RESOURCES, INC.',
 'BREITBURN ENERGY PARTNERS LP',
 'BRISTOW GROUP INC',
 'C&J ENERGY SERVICES LTD.',
 'CAESARS ENTERTAINMENT CORP',
 'CAL DIVE INTERNATIONAL, INC.',
 'CASTLE A M & CO',
 'CENVEO, INC',
 'CHAPARRAL ENERGY, INC.',
 'CHC GROUP LTD.',
 'CIBER INC',
 'CLAIRES STORES INC',
 'CLOUD PEAK ENERGY INC.',
 'COBALT INTERNATIONAL ENERGY, INC.',
 'CUMULUS MEDIA INC',
 'DEAN FOODS CO',
 'DENV

In [73]:
# compare the two sets 

brd_names.difference(company_unique)

{'BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC',
 'CAL DIVE INTERNATIONAL, INC.',
 'STANDARD REGISTER CO',
 'VENOCO, INC.'}

In [78]:
query = """
SELECT distinct company_name, period_end_date
FROM sec_all_raw
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%BLACK ELK ENERGY%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company

#black_elk filed for bankruptcy on 8/11/2015  
# has 10-Q 2014-09-30

,company_name,period_end_date
0,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2010-12-31
1,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2011-12-31
2,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2012-03-31
3,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2012-06-30
4,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2012-09-30
5,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2012-12-31
6,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2013-03-31
7,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2013-06-30
8,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2013-09-30
9,"BLACK ELK ENERGY OFFSHORE OPERATIONS, LLC",2013-12-31


In [44]:
query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%CAL DIVE INTERNATIONAL, INC.%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company

# filed 3/3/2015 
# 2014 annual report would have been due in April 2015
# company filed for bankruptcy before the annual report due
# but did they file quarterly reports? 
# yes, turned in a 2014-09-30 10-Q

,company_name,form,number_of_quarters,period_end_date
0,"CAL DIVE INTERNATIONAL, INC.",10-K,0,2010-12-31
1,"CAL DIVE INTERNATIONAL, INC.",10-K,0,2011-12-31
2,"CAL DIVE INTERNATIONAL, INC.",10-K,0,2012-12-31
3,"CAL DIVE INTERNATIONAL, INC.",10-K,0,2013-12-31
4,"CAL DIVE INTERNATIONAL, INC.",10-K,1,2012-03-31
5,"CAL DIVE INTERNATIONAL, INC.",10-K,1,2012-06-30
6,"CAL DIVE INTERNATIONAL, INC.",10-K,1,2012-09-30
7,"CAL DIVE INTERNATIONAL, INC.",10-K,1,2012-12-31
8,"CAL DIVE INTERNATIONAL, INC.",10-K,1,2013-03-31
9,"CAL DIVE INTERNATIONAL, INC.",10-K,1,2013-06-30


In [56]:

query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%STANDARD REGISTER%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company


## have quarterly report fall 2014, bankruptcy 03/12/2015

,company_name,form,number_of_quarters,period_end_date
0,STANDARD REGISTER CO,10-K,0,2010-12-31
1,STANDARD REGISTER CO,10-K,0,2011-12-31
2,STANDARD REGISTER CO,10-K,0,2012-12-31
3,STANDARD REGISTER CO,10-K,0,2013-12-31
4,STANDARD REGISTER CO,10-K,4,2011-12-31
5,STANDARD REGISTER CO,10-K,4,2012-12-31
6,STANDARD REGISTER CO,10-K,4,2013-12-31


In [57]:



query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%SUNEDISON, INC%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company

# bankruptcy 4/21/2016
# have 10-Q 	2015-09-30

,company_name,form,number_of_quarters,period_end_date
0,"SUNEDISON, INC.",10-K,0,2010-12-31
1,"SUNEDISON, INC.",10-K,0,2011-12-31
2,"SUNEDISON, INC.",10-K,0,2012-12-31
3,"SUNEDISON, INC.",10-K,0,2013-12-31
4,"SUNEDISON, INC.",10-K,0,2014-12-31
5,"SUNEDISON, INC.",10-K,4,2011-12-31
6,"SUNEDISON, INC.",10-K,4,2012-12-31
7,"SUNEDISON, INC.",10-K,4,2013-12-31
8,"SUNEDISON, INC.",10-K,4,2014-12-31


In [76]:

query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%VENOCO%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company
#bankruptcy 3/18/2016  - some filings are VENOCO other filings are DENVER PARENT CORP
#BRD list has Venoco, Inc. (Denver Parent Corp.)
    

,company_name,form,number_of_quarters,period_end_date
0,"VENOCO, INC.",10-K,0,2010-12-31
1,"VENOCO, INC.",10-K,0,2011-12-31
2,"VENOCO, INC.",10-K,0,2012-12-31
3,"VENOCO, INC.",10-K,0,2013-12-31
4,"VENOCO, INC.",10-K,1,2012-03-31
5,"VENOCO, INC.",10-K,1,2012-06-30
6,"VENOCO, INC.",10-K,1,2012-09-30
7,"VENOCO, INC.",10-K,1,2012-12-31
8,"VENOCO, INC.",10-K,1,2013-03-31
9,"VENOCO, INC.",10-K,1,2013-06-30


In [75]:

query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%DENVER PARENT%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company
 #bankruptcy 3/18/2016  - some filings are VENOCO

,company_name,form,number_of_quarters,period_end_date
0,DENVER PARENT CORP,10-K,0,2011-12-31
1,DENVER PARENT CORP,10-K,0,2012-12-31
2,DENVER PARENT CORP,10-K,0,2013-12-31
3,DENVER PARENT CORP,10-K,0,2014-12-31
4,DENVER PARENT CORP,10-K,0,2015-12-31
5,DENVER PARENT CORP,10-K,1,2013-03-31
6,DENVER PARENT CORP,10-K,1,2013-06-30
7,DENVER PARENT CORP,10-K,1,2013-09-30
8,DENVER PARENT CORP,10-K,1,2013-12-31
9,DENVER PARENT CORP,10-K,1,2014-03-31
